In [1]:
import numpy as np
from math import *
from scipy import integrate
from cyfuncs import zhao_func
from exceptions import Exception, ValueError, OverflowError, ZeroDivisionError

In this notebook I integrate the DM squared density (to obtain the *J-factor*) using the same strategy as in CLUMPY:  
I split the integral into three regions:  
- the "front" part of the Halo, zone 1   
- the "rear" part of the Halo, zone 2  
- the inner cusp/core, zone 3  

In [2]:
# physical parameters as in Alex's simulations (except rt)
rh = 0.04
D = 40.
r0 = 0.35
rho0 = 7.55e7
rt = 2.#np.inf
rc = 0.01
theta = 2*rh/D
rhosat = 1e19

theta = 0.5
Dprime = D/r0
rtprime = rt/r0
rcprime = rc/r0
ymin = cos( np.radians(theta) )
Amin = cos( np.radians( atan( rc/D ) ) )

a = 1.
b = 3.
c = 1.

These functions implement the generalised NFW (as in the module `cyfuncs.pyx`)

In [3]:
from itertools import product

In [4]:
r0 = [-3, 2]
a = [0.5, 3]
b = [2, 6]
c = [0, 1.2]

def density(x, a, b, c, r0):
    if c>1e-5:
        if x > rc/r0:
            return 2*zhao_func(x, a, b, c)
        else:
            return rhosat
    else:
        return 2*zhao_func(x, a, b, 0.)

def radius(z, y):
    try:
        return sqrt( z*z + Dprime*Dprime * (1 - y*y ) )
    except (OverflowError, ZeroDivisionError, ValueError):
        return np.nan


def integrand(x, y, ymin, rt, D, a, b, c, r0):
    try:
        return density( radius(x, y), a, b, c, r0 )**2
    except (OverflowError, ZeroDivisionError):
        return np.nan

def lim_l(y, ymin, rt, D, a, b, c, r0):
    l_min = 0#- sqrt( rt*rt - D*D * (1. - y*y) )
    l_max = + sqrt( rt*rt - D*D * (1. - y*y) )
    return [l_min, l_max]


def lim_y(ymin, rt, D, a, b, c, r0):
    return [ymin, 1.]

In [20]:
print '%5s %5s %5s %5s %10s %10s %8s %8s'%('a', 'b', 'c', 'r0', 'I', 'Ierr', 'I>Ierr', 'DIerr')

for p in product( range(2), repeat=4 ):
    I = integrate.nquad(integrand, ranges=[lim_l, lim_y], 
                        args=(ymin, np.inf, D/r0[p[0]], a[p[1]], b[p[2]], c[p[3]], r0[p[0]]), 
                        opts=[{'limit':1000, 'epsabs':1.e-6, 'epsrel':1.e-8},
                              {'limit':1000, 'epsabs':1.e-6, 'epsrel':1.e-8}])

    print '%5.2f %5.2f %5.2f %5.2f %10.2e %10.2e %8s %10.3e'%(r0[p[0]], a[p[1]], b[p[2]], c[p[3]], I[0], I[1], I[0]>I[1], round(I[1]/I[0],3))

    a     b     c    r0          I       Ierr   I>Ierr    DIerr
-3.00  0.50  2.00  0.00   1.44e-06   5.21e-07     True  3.600e-01
-3.00  0.50  2.00  1.20   1.71e-04   1.38e-05     True  8.100e-02
-3.00  0.50  6.00  0.00   9.68e-10   3.86e-07    False  3.990e+02
-3.00  0.50  6.00  1.20   1.72e-05   3.39e-04    False  1.967e+01
-3.00  3.00  2.00  0.00   1.02e-04   9.68e-07     True  9.000e-03
-3.00  3.00  2.00  1.20   5.30e-04   1.98e-06     True  4.000e-03
-3.00  3.00  6.00  0.00   3.19e-05   6.40e-07     True  2.000e-02
-3.00  3.00  6.00  1.20   3.84e-04   1.94e-06     True  5.000e-03
 2.00  0.50  2.00  0.00   1.44e-06   5.21e-07     True  3.600e-01
 2.00  0.50  2.00  1.20   1.70e-04   9.64e+26    False  5.663e+30
 2.00  0.50  6.00  0.00   9.68e-10   3.86e-07    False  3.990e+02
 2.00  0.50  6.00  1.20   3.18e+26   4.61e+27    False  1.451e+01
 2.00  3.00  2.00  0.00   1.02e-04   9.68e-07     True  9.000e-03
 2.00  3.00  2.00  1.20   5.30e-04   1.98e-06     True  4.000e-03
 2.00  3.00 